In [80]:
cat > ./model_kmer_sim/model.stan << EOF
data {
    int <lower=1> N_DATA;
    int <lower=1> N_FRAG;
    int <lower=1> N_SEG;
    int <lower=1> N_SEG_PER_FRAG;
    int <lower=1> N_REP;
    
    int <lower=0> OUTPUT[N_DATA];
    int <lower=1> INPUT[N_DATA];
    real <lower=0> PHI;
}

parameters {
    real<lower=0> SEG[N_SEG];
}

model {
    //print("hello");
    //for (j in 1:N_SEG) {
    //    print(SEG[j]);
    //}
    
    // Shrinkage priors on each effect size
    for (j in 1:N_SEG) {
        SEG[j] ~ lognormal(0, 0.3);
    }
    
    // Likelihood
    for (i in 1:N_FRAG) {
    
        // gather effect from corresponding segments
        real seg_effect = 1;
        for (j in i:(i + N_SEG_PER_FRAG - 1)) {
            seg_effect *= SEG[j];
        }
        
        // mean(output) := segment effect x input
        int idx_data;
        for (k in 1:N_REP) {
            idx_data = N_FRAG * (i - 1) + k;
            OUTPUT[idx_data] ~ neg_binomial_2(INPUT[idx_data] * seg_effect, PHI);
            
            //print(seg_effect, " ", INPUT[idx_data], " ", INPUT[idx_data] * seg_effect);
        } 
    }
}
EOF

In [81]:
cat ./model_kmer_sim/model.stan

data {
    int <lower=1> N_DATA;
    int <lower=1> N_FRAG;
    int <lower=1> N_SEG;
    int <lower=1> N_SEG_PER_FRAG;
    int <lower=1> N_REP;
    
    int <lower=0> OUTPUT[N_DATA];
    int <lower=1> INPUT[N_DATA];
    real <lower=0> PHI;
}

parameters {
    real<lower=0> SEG[N_SEG];
}

model {
    //print("hello");
    //for (j in 1:N_SEG) {
    //    print(SEG[j]);
    //}
    
    // Shrinkage priors on each effect size
    for (j in 1:N_SEG) {
        SEG[j] ~ lognormal(0, 0.3);
    }
    
    // Likelihood
    for (i in 1:N_FRAG) {
    
        // gather effect from corresponding segments
        real seg_effect = 1;
        for (j in i:(i + N_SEG_PER_FRAG - 1)) {
            seg_effect *= SEG[j];
        }
        
        // mean(output) := segment effect x input
        int idx_data;
        for (k in 1:N_REP) {
            idx_data = N_FRAG * (i - 1) + k;
            OUTPUT[idx_data] ~ neg_binomial_2(INPUT[idx_data] * seg_effect, PHI);
            
            //print(seg_effe

In [82]:
source config.sh
FD_CUR=$(pwd)
STAN_PATH=${FD_BASE}/exe/cmdstan

make -C ${STAN_PATH} ${FD_CUR}/model_kmer_sim/model

make: Entering directory '/work/kk319/exe/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  --o=/hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/model_kmer_sim/model.hpp /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/model_kmer_sim/model.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include   -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_5.7.0/include    -DBOOST_DISABLE_ASSERTS         -c -Wno-ignored-attributes   -x c++ -o /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/model_kmer_sim/model.o /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/model_kmer_sim/model.hpp
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/s

In [84]:
cat ./model_kmer_sim/input_trivial_v1.json

{"N_DATA":100,"N_FRAG":10,"N_SEG":15,"N_SEG_PER_FRAG":5,"N_REP":10,"OUTPUT":[2,5,1,3,4,1,4,2,5,5,0,1,3,2,3,6,4,2,2,3,3,2,3,1,2,5,4,3,6,2,3,3,3,3,5,5,6,6,1,3,1,1,3,4,2,2,2,3,4,2,4,3,5,1,3,1,3,2,2,4,5,2,4,2,4,1,4,2,4,1,8,0,0,1,2,6,9,5,1,3,2,0,2,1,1,3,7,0,0,5,2,2,1,1,0,7,3,1,4,3],"INPUT":[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3],"PHI":5,"SEG_TRUE":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]}


In [85]:
cat ./model_kmer_sim/init.json

{"SEG":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]}


In [70]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v1.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v1.json \
    output file=${FD_WRK}/output_trivial_v1.csv
EOF

Submitted batch job 11380474


In [86]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v21.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v21.json \
    output file=${FD_WRK}/output_trivial_v21.csv
EOF

Submitted batch job 11382199


In [87]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v22.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=100 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v22.json \
    output file=${FD_WRK}/output_trivial_v22.csv
EOF

Submitted batch job 11382200


In [92]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v31.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=1000 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v31.json \
    output file=${FD_WRK}/output_trivial_v31.csv
EOF

Submitted batch job 11382350


In [93]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v32.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=1000 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v32.json \
    output file=${FD_WRK}/output_trivial_v32.csv
EOF

Submitted batch job 11382351


In [88]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v41.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=1000 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v41.json \
    output file=${FD_WRK}/output_trivial_v41.csv
EOF

Submitted batch job 11382216


In [89]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_trivial_v42.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=1000 num_warmup=1000 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_trivial_v42.json \
    output file=${FD_WRK}/output_trivial_v42.csv
EOF

Submitted batch job 11382217


In [45]:
### set environment on DCC
FD_WRK=./model_kmer_sim
FD_LOG=${FD_WRK}/log
mkdir -p ${FD_LOG}

sbatch -p scavenger \
    --mem 8G \
    -o ${FD_LOG}/model_null.txt \
    <<'EOF'
#!/bin/bash
FD_WRK=./model_kmer_sim
${FD_WRK}/model sample \
    thin=1 num_samples=10 num_warmup=10 \
    init=${FD_WRK}/init.json \
    data   file=${FD_WRK}/input_null.json \
    output file=${FD_WRK}/output_null.csv
EOF

Submitted batch job 11352221
